In [1]:
import pandas as pd
import yfinance as yf
import requests
import numpy as np
from scipy.optimize import minimize

In [2]:
class stock_fundemental_analysis:
    def __init__(self,ticker):
        self.ticker = ticker
        self.url = f'https://finance.yahoo.com/quote/{ticker}?p={ticker}'
        self.years = 5
        
        if not self._is_valid_ticker():
            raise ValueError("Invalid ticker")
        
        
    def _is_valid_ticker(self):
        try:
            return bool(yf.Ticker(self.ticker).info)
        except yf.utils.exceptions.YahooFinanceError:
            return False
        
        
    # Decorator to fetch balance sheet data
    def decorator_balance_sheet(func):
        def wrap_func(self,*args,**kwargs):
            api_key = open('API.txt','r').read()
            balance_sheet = requests.get(f"http://financialmodelingprep.com/api/v3/balance-sheet-statement/{self.ticker}?limit={self.years}&apikey={api_key}")
            balance_sheet =balance_sheet.json()
            func(self,balance_sheet,*args,**kwargs)
        return wrap_func
   
    # Decorater to print the fundamenta data
    def decorator(func):
        def wrap_func(*arg,**kwarg):
            hi = func(*arg,**kwarg)
            print(hi[1])
        return wrap_func
    
    
    #Function to fetch share price from yahoo finance
    def share_price_data(self):
        stock_data = yf.download(self.ticker, start='2000-04-18', end='2023-05-29')['Adj Close']
        return pd.DataFrame(stock_data)
        
    @decorator
    def fundamental_data(self):
        r = requests.get(self.url,headers ={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'})
        data = pd.read_html(r.text)
        return data
    
    # Function to analyse balance sheet data
    @decorator_balance_sheet
    def balance_sheet(self,balance_sheet):
        
        total_current_asset = balance_sheet[0]["totalCurrentAssets"]/1000000000
        total_current_liabilities = balance_sheet[0]["totalCurrentLiabilities"]/1000000000
        
        
        total_debt = balance_sheet[0]['totalDebt']
        cash_and_equivalents = balance_sheet[0]['cashAndCashEquivalents']
        
        cash_debt_difference = (cash_and_equivalents - total_debt)/1000000000000
        
        print(f"total current asset of {self.ticker} : {total_current_asset} Billion")
        print(f"total current liabilities of {self.ticker} : {total_current_liabilities} Billion")
        print(f"Cash Debt Difference : {cash_debt_difference :,} Billion")
        

In [3]:
A = stock_fundemental_analysis("TSM")

In [4]:
b = A.fundamental_data()

                          0             1
0                Market Cap      511.328B
1         Beta (5Y Monthly)          1.24
2            PE Ratio (TTM)         15.75
3                 EPS (TTM)          6.26
4             Earnings Date           NaN
5  Forward Dividend & Yield  1.79 (1.75%)
6          Ex-Dividend Date  Jun 15, 2023
7             1y Target Est        102.71


In [5]:
A.balance_sheet()

total current asset of TSM : 2052.896744 Billion
total current liabilities of TSM : 944.226817 Billion
Cash Debt Difference : 0.454639611 Billion


In [6]:
""" Objective function for optimization problem"""
def risk_parity(weights,cov_matrix):
    
    portfolio_variance = np.dot(weights.T,np.dot(cov_matrix,weights))
    risk_contributions = np.dot(cov_matrix,weights)/ np.sqrt(portfolio_variance)
    risk_parity_score = np.sum((risk_contributions - np.mean(risk_contributions)) ** 2)
    return risk_parity_score

class allocation(stock_fundemental_analysis):
    def __init__(self,ticker):
        super().__init__(ticker)
        
        if not yf.Ticker(ticker).info:
            raise ValueError("Give a valid ticker symbol")
    
    # return of equity
    def price_returns(self):
        try:
            share_price = super().share_price_data()
            returns = share_price.pct_change().dropna()
            return returns
        except Exception as e:
            raise ValueError("Error in calculating price returns:",str(e))
    
    
    # optimising the weights for max return
    def optimisation(self): 
        try:
            price_returns =self.price_returns()
            cov_matrix =  price_returns.cov()
            count =len(price_returns.columns)
            constraints = ({'type':'eq', 'fun': lambda x: np.sum(x)-1})
            bounds = [(0,1)]*count
            initial_weights = np.ones(count)/count
            np.random.seed(34)
            result = minimize(risk_parity, initial_weights, args=(cov_matrix,), method='SLSQP', constraints=constraints, bounds=bounds)
            optimal_weights = result.x
            return optimal_weights
        except Exception as e:
            raise ValueError("Error in optimisation",str(e))

In [7]:
a = allocation('AAPL KO TSLA TSM')
optimal_weights = np.round(a.optimisation(),9)*100
print("Optimal Weights:", optimal_weights)

[*********************100%***********************]  4 of 4 completed
Optimal Weights: [33.6923921 32.2016906  0.        34.1059172]
